In [1]:
from x_xy.subpkgs import omc
import qmt
import json
import joblib
import tree
import numpy as np

output_file_ending = ""

_marker_closest_to_rigid_imu = {
    "seg1": 4, "seg5": 4, "seg2": 3, "seg3": 3, "seg4": 2
}

def _segment_names_in_experiment(exp_id: str) -> list[str]:
    all = set(["seg1", "seg2", "seg3", "seg4", "seg5"])
    
    if exp_id == "S_04":
        all -= set(["seg1", "seg5"])
    elif exp_id == "D_01":
        all -= set(["seg3"])
    else:
        pass

    return list(all)

def _imu_names_setup_file(exp_id: str) -> list[str]:
    if exp_id == "D_01":
        return ["imu_rigid"]
    return ["imu_rigid", "imu_flex"]

def _get_alignment(data: dict, exp_id: str, hz_omc, hz_imu):
    hz_alignment = 120.0
    hz_in = omc.hz_helper(list(data.keys()), hz_imu=hz_imu, 
                          hz_omc=hz_omc, imus=_imu_names_setup_file(exp_id))
    data = omc.resample(data, hz_in, hz_alignment, vecinterp_method="cubic")
    data = omc.crop_tail(data, hz_alignment)

    acc, gyr, mag, q, pos, names = [], [], [], [], [], []
    for seg_name, seg_data in data.items():
        imu = seg_data["imu_rigid"]
        acc.append(imu["acc"])
        gyr.append(imu["gyr"])
        mag.append(imu["mag"])
        q.append(seg_data["quat"])
        pos.append(seg_data[f"marker{_marker_closest_to_rigid_imu[seg_name]}"])
        names.append(seg_name)

    info = qmt.alignOptImu(gyr, acc, mag, q, pos, rate=hz_alignment, names=names, params=dict(fast=True))

    def _np_tolist(leaf):
        if isinstance(leaf, np.ndarray):
            return leaf.tolist()
        return leaf
    
    with open(f"alignment_infos/alignment_info_{exp_id}{output_file_ending}.json", "w") as file:
        json.dump(tree.map_structure(_np_tolist, info), file, indent=1)
    joblib.dump(info, f"alignment_infos/alignment_info_{exp_id}{output_file_ending}.joblib")

    qEOpt2EImu_euler_deg = info["qEOpt2EImu_euler_deg"]
    qImu2Seg_euler_deg = {seg_name: dict() for seg_name in data}
    for seg_name in data:
        qImu2Seg_euler_deg[seg_name]["imu_rigid"] = info[f"qImu2Seg_{seg_name}_euler_deg"]
    print(qImu2Seg_euler_deg)
    return qEOpt2EImu_euler_deg, qImu2Seg_euler_deg


def to_joblib(exp_id: str, hz_omc: int):

    folder = f"synced_aligned_nan_croptail/synced_aligned_nan_croptail_{exp_id}{output_file_ending}.joblib"
    path_optitrack_file = f"/Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/{exp_id}/optitrack/{exp_id}_{hz_omc}Hz.csv"
    path_imu_folder = f"/Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/{exp_id}/imu"
    setup_file = "/Users/simon/Documents/PYTHON/x_xy_v2/x_xy/subpkgs/exp/setups/setup.json"

    hz_omc = float(hz_omc)
    hz_imu = float(omc.utils.autodetermine_imu_freq(path_imu_folder))
    
    # perform sync
    data, imu_sync_offset = omc.read_omc(
        path_marker_imu_setup_file=setup_file,
        path_optitrack_file=path_optitrack_file,
        path_imu_folder=path_imu_folder,
        imu_names_setup_file=_imu_names_setup_file(exp_id),
        segment_names_setup_file=_segment_names_in_experiment(exp_id)
        )

    qEOpt2EImu_euler_deg, qImu2Seg_euler_deg = _get_alignment(data, exp_id, hz_omc, hz_imu)
    
    # aligned
    data, _ = omc.read_omc(
        path_marker_imu_setup_file="/Users/simon/Documents/PYTHON/x_xy_v2/x_xy/subpkgs/exp/setups/setup.json",
        path_optitrack_file=path_optitrack_file,
        path_imu_folder=path_imu_folder,
        imu_sync_offset=imu_sync_offset,
        qEOpt2EImu_euler_deg=qEOpt2EImu_euler_deg,
        qImu2Seg_euler_deg=qImu2Seg_euler_deg,
        imu_names_setup_file=_imu_names_setup_file(exp_id),
        segment_names_setup_file=_segment_names_in_experiment(exp_id)
        )
    
    hz_in = omc.hz_helper(list(data.keys()), hz_imu=hz_imu, 
                          hz_omc=hz_omc, imus=_imu_names_setup_file(exp_id))
    # croptail
    data = omc.crop_tail(data, hz_in, strict=False)

    joblib.dump(data, folder)

    return data


In [2]:
to_joblib("S_04", 30)

OMC Hz: 30
IMU Hz: 100
IMU File Prefix: MT_0120041E-000-000_00B483
IMU File Delimiter: ;
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_04/optitrack/S_04_30Hz.csv
Segment: 2, IMU: A3, Offset: 338
Segment: 2, IMU: 84, Offset: 339
Segment: 3, IMU: A0, Offset: 338
Segment: 3, IMU: A2, Offset: 339
Segment: 4, IMU: A4, Offset: 338
Segment: 4, IMU: 9B, Offset: 339
`crop_tail`: Crop off at t=647.3666666666667.
{'seg2': {'imu_rigid': array([ 0.092,  1.058, -0.424])}, 'seg3': {'imu_rigid': array([-0.156,  1.187, -0.206])}, 'seg4': {'imu_rigid': array([ 0.217,  0.528, -0.082])}}
OMC Hz: 30
IMU Hz: 100
IMU File Prefix: MT_0120041E-000-000_00B483
IMU File Delimiter: ;
Segment: 2, IMU: A3, Offset: 338
Segment: 2, IMU: 84, Offset: 339
Segment: 3, IMU: A0, Offset: 338
Segment: 3, IMU: A2, Offset: 339
Segment: 4, IMU: A4, Offset: 338
Segment: 4, IMU: 9B, Offset: 339
`crop_tail`: Crop off at t=647.3666666666667.


{'seg2': {'imu_rigid': {'acc': array([[ 0.27261133, -0.17058866,  9.68719989],
          [ 0.24050684, -0.16959634,  9.70061057],
          [ 0.27200275, -0.17152373,  9.68720107],
          ...,
          [ 0.01851525, -0.12527333,  9.68743387],
          [ 0.03027172, -0.15607999,  9.69821096],
          [ 0.05458702, -0.15781183,  9.72238394]]),
   'gyr': array([[-0.00540496,  0.00398051,  0.00511017],
          [-0.00432312,  0.00254506,  0.00236733],
          [-0.00486403,  0.00326329,  0.00373925],
          ...,
          [-0.00185983,  0.00112892,  0.00028699],
          [-0.00427073,  0.00480142,  0.00280068],
          [-0.00507538,  0.00122842,  0.00354523]]),
   'mag': array([[-0.04418586,  0.04589746, -0.61683573],
          [-0.0456982 ,  0.03669054, -0.60697147],
          [-0.04147745,  0.04283268, -0.60269914],
          ...,
          [-0.0330369 ,  0.04593172, -0.64780884],
          [-0.02984899,  0.03763605, -0.64780603],
          [-0.02904206,  0.0359573 , -0.64

In [3]:
to_joblib("S_06", 30)

OMC Hz: 30
IMU Hz: 40
IMU File Prefix: MT_0120041E-000-000_00B483
IMU File Delimiter: ;
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_06/optitrack/S_06_30Hz.csv
Segment: 2, IMU: A3, Offset: 156
Segment: 2, IMU: 84, Offset: 156
Segment: 3, IMU: A0, Offset: 156
Segment: 3, IMU: A2, Offset: 156
Segment: 4, IMU: A4, Offset: 156
Segment: 4, IMU: 9B, Offset: 156
Segment: 1, IMU: 9F, Offset: 156
Segment: 1, IMU: B8, Offset: 156
Segment: 5, IMU: 99, Offset: 156
Segment: 5, IMU: 85, Offset: 156
`crop_tail`: Crop off at t=401.0.
{'seg2': {'imu_rigid': array([-0.276,  0.323, -0.213])}, 'seg3': {'imu_rigid': array([-0.101,  0.403, -0.218])}, 'seg4': {'imu_rigid': array([ 0.298,  0.044, -0.103])}, 'seg1': {'imu_rigid': array([0.141, 0.151, 0.298])}, 'seg5': {'imu_rigid': array([ 0.266,  0.186, -1.048])}}
OMC Hz: 30
IMU Hz: 40
IMU File Prefix: MT_0120041E-000-000_00B483
IMU File Delimiter: ;
Segment: 2, IMU: A3, Offset: 156
Segment: 2, IMU: 84, Offse

{'seg2': {'imu_rigid': {'acc': array([[-0.24394056, -1.01986417,  9.66154746],
          [-0.21422201, -1.04108447,  9.63723616],
          [-0.23124514, -1.04567182,  9.66658202],
          ...,
          [-0.12876268, -1.81402825,  9.53563998],
          [-0.14037535, -1.82008315,  9.52797882],
          [-0.12901484, -1.82022588,  9.54835857]]),
   'gyr': array([[-0.00270493,  0.00189018,  0.00461843],
          [-0.00720878,  0.00259346,  0.00264726],
          [-0.00546091,  0.00344788,  0.00312723],
          ...,
          [-0.00780156,  0.00333474,  0.00329789],
          [-0.00367856,  0.00299691,  0.00463085],
          [-0.00527051,  0.00211949,  0.00273605]]),
   'mag': array([[-0.01244536,  0.29326853,  1.17967902],
          [-0.01076594,  0.2922681 ,  1.17527911],
          [-0.00856018,  0.2932362 ,  1.17599508],
          ...,
          [-0.26433779,  0.04936922,  1.18346867],
          [-0.26228059,  0.05502517,  1.19710848],
          [-0.26325643,  0.05211084,  1.19

In [4]:
to_joblib("S_07", 30)

OMC Hz: 30
IMU Hz: 40
IMU File Prefix: MT_0120041E-000-000_00B483
IMU File Delimiter: ;
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_07/optitrack/S_07_30Hz.csv
Segment: 2, IMU: A3, Offset: 112
Segment: 2, IMU: 84, Offset: 112
Segment: 3, IMU: A0, Offset: 112
Segment: 3, IMU: A2, Offset: 112
Segment: 4, IMU: A4, Offset: 112
Segment: 4, IMU: 9B, Offset: 112
Segment: 1, IMU: 9F, Offset: 112
Segment: 1, IMU: B8, Offset: 112
Segment: 5, IMU: 99, Offset: 112
Segment: 5, IMU: 85, Offset: 112
`crop_tail`: Crop off at t=387.43333333333334.
{'seg2': {'imu_rigid': array([-0.375,  0.37 , -0.132])}, 'seg3': {'imu_rigid': array([-0.411,  0.305, -0.147])}, 'seg4': {'imu_rigid': array([-0.129,  0.041, -0.205])}, 'seg1': {'imu_rigid': array([-0.389,  0.101,  0.542])}, 'seg5': {'imu_rigid': array([ 0.078,  0.327, -0.902])}}
OMC Hz: 30
IMU Hz: 40
IMU File Prefix: MT_0120041E-000-000_00B483
IMU File Delimiter: ;
Segment: 2, IMU: A3, Offset: 112
Segment: 2

{'seg2': {'imu_rigid': {'acc': array([[-0.21586749, -0.39481148,  9.71938255],
          [-0.22135694, -0.4219745 ,  9.71234534],
          [-0.22485066, -0.41443268,  9.70879382],
          ...,
          [-0.09169674,  1.87805918,  9.54966471],
          [-0.10446031,  1.86201694,  9.53959237],
          [-0.08933394,  1.88557414,  9.54107222]]),
   'gyr': array([[-0.00452571,  0.00282553,  0.00386899],
          [-0.00290748,  0.00209743,  0.00277414],
          [-0.00673461,  0.00370535,  0.00316329],
          ...,
          [-0.00563371,  0.00285422,  0.00276407],
          [-0.00458383,  0.00214039,  0.00191387],
          [-0.00483863,  0.00397431,  0.0028744 ]]),
   'mag': array([[0.60330012, 0.50290056, 0.73838312],
          [0.59909885, 0.50485859, 0.72864065],
          [0.60032914, 0.50241849, 0.7325443 ],
          ...,
          [0.10253407, 0.46147889, 1.15652288],
          [0.0986332 , 0.46540318, 1.15336518],
          [0.10300796, 0.46390615, 1.15187521]])},
  'imu

In [5]:
to_joblib("S_08", 30)

OMC Hz: 30
IMU Hz: 40
IMU File Prefix: MT_0120041E-000-000_00B483
IMU File Delimiter: ;
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_08/optitrack/S_08_30Hz.csv
Segment: 2, IMU: A3, Offset: 124
Segment: 2, IMU: 84, Offset: 124
Segment: 3, IMU: A0, Offset: 124
Segment: 3, IMU: A2, Offset: 125
Segment: 4, IMU: A4, Offset: 124
Segment: 4, IMU: 9B, Offset: 124
Segment: 1, IMU: 9F, Offset: 124
Segment: 1, IMU: B8, Offset: 124
Segment: 5, IMU: 99, Offset: 124
Segment: 5, IMU: 85, Offset: 124
`crop_tail`: Crop off at t=310.3.
{'seg2': {'imu_rigid': array([-0.117,  0.376, -0.944])}, 'seg3': {'imu_rigid': array([-0.054,  0.608, -0.878])}, 'seg4': {'imu_rigid': array([ 0.385,  0.556, -0.228])}, 'seg1': {'imu_rigid': array([-1.033, -0.204, -0.995])}, 'seg5': {'imu_rigid': array([ 0.236,  0.16 , -1.826])}}
OMC Hz: 30
IMU Hz: 40
IMU File Prefix: MT_0120041E-000-000_00B483
IMU File Delimiter: ;
Segment: 2, IMU: A3, Offset: 124
Segment: 2, IMU: 84, Of

{'seg2': {'imu_rigid': {'acc': array([[-0.19772294,  1.36163008,  9.60553874],
          [-0.20251596,  1.37744352,  9.6346136 ],
          [-0.20903953,  1.36937644,  9.63142192],
          ...,
          [-0.09203624,  2.07786754,  9.52045583],
          [-0.08922317,  2.09392898,  9.50829597],
          [-0.11010659,  2.06951333,  9.50843764]]),
   'gyr': array([[-0.0053276 ,  0.00285769,  0.00406673],
          [-0.00529196,  0.00340285,  0.00140511],
          [-0.00392626,  0.0017402 ,  0.00263266],
          ...,
          [-0.00781139,  0.003884  ,  0.00287503],
          [-0.00499018,  0.00232402,  0.00109583],
          [-0.0052117 ,  0.00283824,  0.0028391 ]]),
   'mag': array([[0.18329713, 0.62087952, 1.0258407 ],
          [0.17965501, 0.6211792 , 1.02879025],
          [0.17897788, 0.62520673, 1.03605356],
          ...,
          [0.79021598, 0.35627888, 0.73171452],
          [0.78654568, 0.36355309, 0.72820043],
          [0.78801009, 0.3616054 , 0.72871091]])},
  'imu

In [6]:
to_joblib("S_09", 120)

OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: ;
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_09/optitrack/S_09_120Hz.csv
Segment: 2, IMU: A3, Offset: 121
Segment: 2, IMU: 84, Offset: 122
Segment: 3, IMU: A0, Offset: 122
Segment: 3, IMU: A2, Offset: 121
Segment: 4, IMU: A4, Offset: 121
Segment: 4, IMU: 9B, Offset: 122
Segment: 1, IMU: 9F, Offset: 121
Segment: 1, IMU: B8, Offset: 122
Segment: 5, IMU: 99, Offset: 122
Segment: 5, IMU: 85, Offset: 121
`crop_tail`: Crop off at t=245.44166666666666.
{'seg2': {'imu_rigid': array([-0.041,  0.518, -0.617])}, 'seg3': {'imu_rigid': array([ 0.013,  0.752, -0.626])}, 'seg4': {'imu_rigid': array([0.562, 0.579, 0.055])}, 'seg1': {'imu_rigid': array([-0.236,  0.036,  0.785])}, 'seg5': {'imu_rigid': array([ 0.03 ,  0.254, -1.084])}}
OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: ;
Segment: 2, IMU: A3, Offset: 121
Segment: 2

{'seg2': {'imu_rigid': {'acc': array([[-0.12702866,  1.50217921,  9.61682648],
          [-0.12115497,  1.51728496,  9.59532764],
          [-0.13396037,  1.50662796,  9.60282193],
          ...,
          [-0.07697744,  1.97228902,  9.52167489],
          [-0.05554353,  1.97505585,  9.53086109],
          [-0.05220255,  1.98950345,  9.53567386]]),
   'gyr': array([[-0.00282269,  0.00309553,  0.00312053],
          [-0.00433754,  0.0021185 ,  0.00322176],
          [-0.00812276,  0.00379123,  0.00297099],
          ...,
          [-0.00570546,  0.0020095 ,  0.00435143],
          [-0.00510829,  0.00269789,  0.00265345],
          [-0.00634608,  0.00364597,  0.00304948]]),
   'mag': array([[ 0.61140871,  0.28736452,  1.00954123],
          [ 0.61046237,  0.28691403,  1.01297298],
          [ 0.60453699,  0.28269227,  1.00599134],
          ...,
          [ 0.72890972, -0.15388347,  1.07866364],
          [ 0.72764432, -0.15954719,  1.07409659],
          [ 0.72861774, -0.15296415,  1.07

In [7]:
to_joblib("S_10", 120)

OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: ;
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_10/optitrack/S_10_120Hz.csv
Segment: 2, IMU: A3, Offset: 107
Segment: 2, IMU: 84, Offset: 108
Segment: 3, IMU: A0, Offset: 107
Segment: 3, IMU: A2, Offset: 107
Segment: 4, IMU: A4, Offset: 107
Segment: 4, IMU: 9B, Offset: 107
Segment: 1, IMU: 9F, Offset: 107
Segment: 1, IMU: B8, Offset: 107
Segment: 5, IMU: 99, Offset: 107
Segment: 5, IMU: 85, Offset: 107
`crop_tail`: Crop off at t=119.84166666666667.
{'seg2': {'imu_rigid': array([-0.598,  0.326, -0.546])}, 'seg3': {'imu_rigid': array([-0.132,  0.412, -0.699])}, 'seg4': {'imu_rigid': array([-0.076,  0.195,  0.01 ])}, 'seg1': {'imu_rigid': array([-0.08 ,  0.305,  0.138])}, 'seg5': {'imu_rigid': array([ 0.02 ,  0.215, -1.382])}}
OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: ;
Segment: 2, IMU: A3, Offset: 107
Segment

{'seg2': {'imu_rigid': {'acc': array([[-0.1222893 ,  1.73033797,  9.57640936],
          [-0.13573754,  1.70422901,  9.59823778],
          [-0.1033593 ,  1.70774823,  9.58333912],
          ...,
          [-0.10333424,  1.90160434,  9.53444925],
          [-0.08823749,  1.87056022,  9.53914012],
          [-0.0827975 ,  1.96178899,  9.5335984 ]]),
   'gyr': array([[-0.00737342,  0.00344265,  0.00301727],
          [-0.00419017,  0.00381195,  0.00357538],
          [-0.00474097,  0.00227692,  0.00309307],
          ...,
          [-0.00585334,  0.00201285,  0.00229596],
          [-0.00542357,  0.00315791,  0.00401774],
          [-0.00327451,  0.00415114,  0.00165075]]),
   'mag': array([[ 0.52428618, -0.02836604,  1.18163249],
          [ 0.53405183, -0.02990739,  1.18427672],
          [ 0.52453719, -0.02715695,  1.18064353],
          ...,
          [ 0.95441485, -0.15233413,  0.83752235],
          [ 0.9554173 , -0.15086096,  0.83945572],
          [ 0.95337489, -0.15986575,  0.84

In [8]:
to_joblib("S_12", 120)

OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: ;
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_12/optitrack/S_12_120Hz.csv
Segment: 2, IMU: A3, Offset: 90
Segment: 2, IMU: 84, Offset: 91
Segment: 3, IMU: A0, Offset: 91
Segment: 3, IMU: A2, Offset: 91
Segment: 4, IMU: A4, Offset: 90
Segment: 4, IMU: 9B, Offset: 92
Segment: 1, IMU: 9F, Offset: 90
Segment: 1, IMU: B8, Offset: 92
Segment: 5, IMU: 99, Offset: 92
Segment: 5, IMU: 85, Offset: 91
`crop_tail`: Crop off at t=330.28333333333336.
{'seg2': {'imu_rigid': array([-0.145,  0.469, -0.919])}, 'seg3': {'imu_rigid': array([ 0.035,  0.553, -0.007])}, 'seg4': {'imu_rigid': array([-0.148,  0.373,  0.207])}, 'seg1': {'imu_rigid': array([ 0.06 , -0.058,  0.08 ])}, 'seg5': {'imu_rigid': array([ 0.047,  0.388, -1.   ])}}
OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: ;
Segment: 2, IMU: A3, Offset: 90
Segment: 2, IMU: 8

{'seg2': {'imu_rigid': {'acc': array([[-2.20038894, -5.50439503,  7.74105936],
          [-2.20471641, -5.51000108,  7.72137556],
          [-2.21481195, -5.52712139,  7.72615861],
          ...,
          [-1.15200883, -6.0469534 ,  7.54310609],
          [-1.16139602, -6.04358909,  7.54294039],
          [-1.15188107, -6.04380487,  7.52854225]]),
   'gyr': array([[-0.00694895,  0.00174327,  0.00426292],
          [-0.00594923,  0.00253499,  0.00226869],
          [-0.00535389,  0.00116958,  0.00334084],
          ...,
          [-0.00416275,  0.00319865,  0.00113719],
          [-0.00683686,  0.00158475,  0.00187915],
          [-0.0051899 ,  0.00220217,  0.0030199 ]]),
   'mag': array([[ 0.70884607,  0.10669241, -0.35134624],
          [ 0.70685892,  0.10492992, -0.35496425],
          [ 0.71759593,  0.10465095, -0.35553619],
          ...,
          [ 0.71517222,  0.13576187, -0.27348229],
          [ 0.71100539,  0.14035501, -0.27694015],
          [ 0.72350932,  0.14091412, -0.27

In [9]:
to_joblib("S_13", 120)

OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: ;
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_13/optitrack/S_13_120Hz.csv
Segment: 2, IMU: A3, Offset: 102
Segment: 2, IMU: 84, Offset: 102
Segment: 3, IMU: A0, Offset: 102
Segment: 3, IMU: A2, Offset: 102
Segment: 4, IMU: A4, Offset: 102
Segment: 4, IMU: 9B, Offset: 102
Segment: 1, IMU: 9F, Offset: 102
Segment: 1, IMU: B8, Offset: 102
Segment: 5, IMU: 99, Offset: 102
Segment: 5, IMU: 85, Offset: 102
`crop_tail`: Crop off at t=396.51666666666665.
{'seg2': {'imu_rigid': array([-0.19 ,  0.182, -0.869])}, 'seg3': {'imu_rigid': array([-0.209,  0.642,  0.152])}, 'seg4': {'imu_rigid': array([-0.327,  0.36 ,  0.127])}, 'seg1': {'imu_rigid': array([-0.193,  0.011, -0.088])}, 'seg5': {'imu_rigid': array([ 0.172,  0.254, -1.075])}}
OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: ;
Segment: 2, IMU: A3, Offset: 102
Segment

{'seg2': {'imu_rigid': {'acc': array([[-1.22663358, -6.03775959,  7.55492922],
          [-1.22808325, -6.02676386,  7.56150104],
          [-1.22818732, -6.06243943,  7.53891035],
          ...,
          [-1.10605265, -6.08822203,  7.54422368],
          [-1.09508142, -6.0860209 ,  7.51677563],
          [-1.10928487, -6.08763   ,  7.51332643]]),
   'gyr': array([[-0.00496213,  0.00261936,  0.00605504],
          [-0.00392676,  0.002821  ,  0.00249421],
          [-0.002669  ,  0.00453235,  0.00125807],
          ...,
          [-0.00468265,  0.00313936,  0.00314391],
          [-0.00542279,  0.00097181,  0.00316315],
          [-0.0063115 ,  0.00186651,  0.00169228]]),
   'mag': array([[ 0.71663292,  0.13755554, -0.27444985],
          [ 0.72222084,  0.13164163, -0.27681953],
          [ 0.7217636 ,  0.13274863, -0.26828881],
          ...,
          [ 0.6129037 ,  0.17155473, -0.453851  ],
          [ 0.60920872,  0.15915808, -0.45096485],
          [ 0.61113291,  0.16237677, -0.46

In [10]:
to_joblib("S_14", 120)

OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: ;
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_14/optitrack/S_14_120Hz.csv
Segment: 2, IMU: A3, Offset: 96
Segment: 2, IMU: 84, Offset: 96
Segment: 3, IMU: A0, Offset: 96
Segment: 3, IMU: A2, Offset: 95
Segment: 4, IMU: A4, Offset: 96
Segment: 4, IMU: 9B, Offset: 96
Segment: 1, IMU: 9F, Offset: 96
Segment: 1, IMU: B8, Offset: 96
Segment: 5, IMU: 99, Offset: 96
Segment: 5, IMU: 85, Offset: 96
`crop_tail`: Crop off at t=300.09166666666664.
{'seg2': {'imu_rigid': array([-0.01 ,  0.481, -0.787])}, 'seg3': {'imu_rigid': array([-0.067,  0.961, -0.139])}, 'seg4': {'imu_rigid': array([-0.085,  0.546,  0.208])}, 'seg1': {'imu_rigid': array([-0.072,  0.129,  0.232])}, 'seg5': {'imu_rigid': array([-0.087,  0.286, -1.054])}}
OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: ;
Segment: 2, IMU: A3, Offset: 96
Segment: 2, IMU: 8

{'seg2': {'imu_rigid': {'acc': array([[-1.06803377, -6.06826846,  7.54168461],
          [-1.06874162, -6.08930712,  7.51367386],
          [-1.07584518, -6.05462844,  7.53602513],
          ...,
          [-0.85537769, -6.19952083,  7.47372583],
          [-0.83531668, -6.19144034,  7.47990821],
          [-0.84149167, -6.17579289,  7.44697089]]),
   'gyr': array([[-0.00491011,  0.00194347,  0.00342198],
          [-0.00548256,  0.00086164,  0.00220149],
          [-0.00201647,  0.00339963,  0.00452783],
          ...,
          [-0.0065794 ,  0.00281593,  0.00373305],
          [-0.00469407,  0.00288098,  0.00090496],
          [-0.00295324,  0.00321287,  0.00432423]]),
   'mag': array([[ 0.60759391,  0.16517512, -0.45078758],
          [ 0.61121033,  0.15899729, -0.45610479],
          [ 0.60267449,  0.17024213, -0.45521144],
          ...,
          [ 0.74105552,  0.11278498, -0.14035746],
          [ 0.74641579,  0.10788473, -0.14155632],
          [ 0.74933074,  0.1146961 , -0.14

In [11]:
to_joblib("S_15", 120)

OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: 	
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_15/optitrack/S_15_120Hz.csv
Segment: 2, IMU: A3, Offset: 118
Segment: 2, IMU: 84, Offset: 118
Segment: 3, IMU: A0, Offset: 118
Segment: 3, IMU: A2, Offset: 118
Segment: 4, IMU: A4, Offset: 118
Segment: 4, IMU: 9B, Offset: 118
Segment: 1, IMU: 9F, Offset: 118
Segment: 1, IMU: B8, Offset: 118
Segment: 5, IMU: 99, Offset: 118
Segment: 5, IMU: 85, Offset: 118
`crop_tail`: Crop off at t=246.175.
{'seg2': {'imu_rigid': array([-0.172,  0.624, -0.483])}, 'seg3': {'imu_rigid': array([-0.116,  1.073, -0.033])}, 'seg4': {'imu_rigid': array([-0.139,  0.41 ,  0.18 ])}, 'seg1': {'imu_rigid': array([-0.227,  0.368,  0.428])}, 'seg5': {'imu_rigid': array([-0.223,  0.269, -0.743])}}
OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: 	
Segment: 2, IMU: A3, Offset: 118
Segment: 2, IMU: 8

{'seg2': {'imu_rigid': {'acc': array([[-2.26887596, -5.76662256,  7.5162687 ],
          [-2.27602083, -5.76713439,  7.5385011 ],
          [-2.29007994, -5.76977595,  7.53406732],
          ...,
          [-1.53611649, -6.04033884,  7.49123331],
          [-1.52942451, -6.05020258,  7.47214928],
          [-1.53998034, -6.04193872,  7.49905666]]),
   'gyr': array([[-0.00283427,  0.00350713,  0.00269774],
          [-0.00684795,  0.00326327,  0.0026346 ],
          [-0.00415617,  0.00196603,  0.00176202],
          ...,
          [-0.00474465,  0.00286747,  0.00386108],
          [-0.00455524,  0.00371246,  0.00237877],
          [-0.00624379,  0.00109349,  0.00205017]]),
   'mag': array([[ 0.6529977 ,  0.11750127,  0.00140666],
          [ 0.6488469 ,  0.11166627,  0.00296611],
          [ 0.65391307,  0.11404153, -0.00321272],
          ...,
          [ 0.40528222, -0.26222673,  0.63113894],
          [ 0.40145571, -0.26605215,  0.63951462],
          [ 0.40120817, -0.26361772,  0.63

In [12]:
to_joblib("S_16", 120)

OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: 	
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_16/optitrack/S_16_120Hz.csv
Segment: 2, IMU: A3, Offset: 155
Segment: 2, IMU: 84, Offset: 155
Segment: 3, IMU: A0, Offset: 155
Segment: 3, IMU: A2, Offset: 156
Segment: 4, IMU: A4, Offset: 155
Segment: 4, IMU: 9B, Offset: 156
Segment: 1, IMU: 9F, Offset: 155
Segment: 1, IMU: B8, Offset: 155
Segment: 5, IMU: 99, Offset: 155
Segment: 5, IMU: 85, Offset: 156
`crop_tail`: Crop off at t=185.0.
{'seg2': {'imu_rigid': array([-0.106,  0.672, -0.483])}, 'seg3': {'imu_rigid': array([ 0.067,  1.283, -0.558])}, 'seg4': {'imu_rigid': array([-0.545, -0.001, -0.196])}, 'seg1': {'imu_rigid': array([-0.686, -0.686,  0.117])}, 'seg5': {'imu_rigid': array([ 0.209,  0.437, -0.718])}}
OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: 	
Segment: 2, IMU: A3, Offset: 155
Segment: 2, IMU: 84,

{'seg2': {'imu_rigid': {'acc': array([[-2.33045467,  1.01387157,  9.4092594 ],
          [-2.33831354,  1.01775393,  9.39472154],
          [-2.32954056,  1.01452831,  9.39577274],
          ...,
          [-3.28345786, -0.05680206,  9.17083249],
          [-3.27689371, -0.05027329,  9.17949541],
          [-3.27651327, -0.05502585,  9.16007288]]),
   'gyr': array([[-0.00441826,  0.00260091,  0.0036864 ],
          [-0.00486833,  0.00298336,  0.00174727],
          [-0.00494214,  0.00324349,  0.00257604],
          ...,
          [-0.005904  ,  0.00519926,  0.00305194],
          [-0.00505884,  0.00127255,  0.00197491],
          [-0.00664954,  0.00521341,  0.00445772]]),
   'mag': array([[-0.56695202, -0.33583485, -0.36482431],
          [-0.56660739, -0.33722635, -0.35602674],
          [-0.57034105, -0.34068466, -0.36156939],
          ...,
          [-0.57033389, -0.44520452, -0.21956185],
          [-0.57171761, -0.43539102, -0.21425656],
          [-0.57351321, -0.4505674 , -0.21

In [13]:
to_joblib("D_01", 120)

OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: 	
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/D_01/optitrack/D_01_120Hz.csv
Segment: 2, IMU: A3, Offset: 133
Segment: 4, IMU: A4, Offset: 133
Segment: 1, IMU: 9F, Offset: 133
Segment: 5, IMU: 99, Offset: 133
`crop_tail`: Crop off at t=474.9166666666667.
{'seg2': {'imu_rigid': array([-0.121,  0.75 , -0.647])}, 'seg4': {'imu_rigid': array([-0.129,  0.332,  0.001])}, 'seg1': {'imu_rigid': array([0.064, 0.344, 0.152])}, 'seg5': {'imu_rigid': array([ 0.529,  0.467, -0.948])}}
OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: 	
Segment: 2, IMU: A3, Offset: 133
Segment: 4, IMU: A4, Offset: 133
Segment: 1, IMU: 9F, Offset: 133
Segment: 5, IMU: 99, Offset: 133
`crop_tail`: Crop off at t=474.9166666666667.


{'seg2': {'imu_rigid': {'acc': array([[-0.39417442, -0.08280917,  9.74018585],
          [-0.40494816, -0.08182837,  9.72659805],
          [-0.39666806, -0.07980152,  9.72721574],
          ...,
          [-0.19675342, -0.11609896,  9.7084004 ],
          [-0.20924261, -0.0927949 ,  9.67208923],
          [-0.26235208, -0.09161399,  9.78116063]]),
   'gyr': array([[-0.00557503,  0.00315707,  0.00326903],
          [-0.00263844,  0.00365147,  0.00168672],
          [-0.00539015,  0.00449902,  0.00204931],
          ...,
          [-0.00368224,  0.00094548,  0.00161087],
          [ 0.00240641,  0.00591161,  0.00265225],
          [ 0.00089029,  0.00440387,  0.00366027]]),
   'mag': array([[ 0.35199004, -0.25117462, -0.58195113],
          [ 0.34896574, -0.25100695, -0.58923845],
          [ 0.35001848, -0.25094302, -0.5833931 ],
          ...,
          [ 0.47783969, -0.24012011, -0.57566793],
          [ 0.47164401, -0.23823686, -0.58293398],
          [ 0.4716562 , -0.24505041, -0.58

In [14]:
to_joblib("T_01", 120)

OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: 	
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/T_01/optitrack/T_01_120Hz.csv
Segment: 2, IMU: A3, Offset: 70
Segment: 2, IMU: 84, Offset: 70
Segment: 3, IMU: A0, Offset: 70
Segment: 3, IMU: A2, Offset: 70
Segment: 4, IMU: A4, Offset: 70
Segment: 4, IMU: 9B, Offset: 70
Segment: 1, IMU: 9F, Offset: 70
Segment: 1, IMU: B8, Offset: 70
Segment: 5, IMU: 99, Offset: 70
Segment: 5, IMU: 85, Offset: 70
`crop_tail`: Crop off at t=216.08333333333334.
{'seg2': {'imu_rigid': array([ 0.176,  0.655, -0.873])}, 'seg3': {'imu_rigid': array([ 0.204,  0.625, -0.209])}, 'seg4': {'imu_rigid': array([-0.093,  0.791,  0.037])}, 'seg1': {'imu_rigid': array([-0.041,  0.462, -0.257])}, 'seg5': {'imu_rigid': array([ 0.219,  0.645, -1.238])}}
OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: 	
Segment: 2, IMU: A3, Offset: 70
Segment: 2, IMU: 8

{'seg2': {'imu_rigid': {'acc': array([[ 2.38077585, -6.77344835,  6.6125056 ],
          [ 2.38183022, -6.78124588,  6.5850687 ],
          [ 2.39020161, -6.78568368,  6.58569629],
          ...,
          [-1.16705429, -6.18945849,  7.43086448],
          [-1.15302664, -6.20299415,  7.42964729],
          [-1.14735118, -6.18131592,  7.43372837]]),
   'gyr': array([[-0.00510884,  0.0026516 ,  0.00268215],
          [-0.00517697,  0.00206273,  0.00251289],
          [-0.00639714,  0.00184274,  0.00165898],
          ...,
          [-0.00470861,  0.00247409,  0.00230224],
          [-0.00569952,  0.00243995,  0.00283213],
          [-0.00404492,  0.00171591,  0.00189218]]),
   'mag': array([[ 0.29776852,  0.28311291, -0.48827069],
          [ 0.303405  ,  0.28928834, -0.48476652],
          [ 0.29559987,  0.28387909, -0.48557121],
          ...,
          [ 0.56212938,  0.11553716, -0.45331361],
          [ 0.55758043,  0.11711449, -0.4449844 ],
          [ 0.56810707,  0.11497984, -0.44